20221010 

In [1]:
import sys,os
sys.path.append(os.path.realpath('..'))

from scripts.functions import dilute, replenish,dilute_species, replenish_species, run_model
from scripts.species import DNA, RNA, Protein, Resource
from scripts.models import Repressor_model


import numpy as np
import scipy.integrate
import scipy.optimize
from bokeh.plotting import figure, output_file, show

import bokeh.plotting
import bokeh.io
from bokeh.models import Span


# We will use Matplotlib to make a 3D plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import panel as pn
pn.extension()

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
#######Define species #######

#T7 RNAP
T7_DNA = DNA()
T7_RNA = RNA()
T7 = Protein()

#Repressor
Repressor_DNA = DNA()
Repressor_RNA = RNA()
Repressor = Protein()

#GFP
GFP_DNA = DNA()
GFP_RNA = RNA()
GFP = Protein()

#Resource
R      = Resource()

#Others
TX = 1
TL = 1

#list of molecules
molecules_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,Repressor_DNA,Repressor_RNA,Repressor,R]


#give index num
for i in range(0,len(molecules_list)):
    molecules_list[i].idx = i




#define values 
##T7
T7_DNA.ic =1
T7_DNA.lc =1

T7.ic = 1
T7.lc = 1

##GFP
GFP_DNA.ic = 1
GFP_DNA.lc = 1

GFP.lc = 0

##Repressor 
Repressor_DNA.ic = 0.2
Repressor_DNA.lc = 0.2


#resources
R.ic =10
R.lc =10

#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


for i in range(0,len(molecules_list)):

    molecules_0[i] = molecules_list[i].ic


In [3]:
#Define parameters
alpha=0.7;
beta=0.07;
K=1.0;
Kd = 0.5;
HC = 1;
KTL=10;
KTX=1;
lamb1=0;
lamb2=0;

parameters_list = (alpha,beta,K,Kd,KTL,KTX,lamb1,lamb2)



In [4]:
#stage 1 kick start
result_all=molecules_0 #initial condition

#DR = 0.2

#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]

run_model(Repressor_model,(1,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)

#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [R]

run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)

#wash out
dilution_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP,GFP_RNA,Repressor_DNA,Repressor,Repressor_RNA,R]
replenish_list = [R]

run_model(Repressor_model,(16,24),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#stage 2 self regeneration 

for n in range (Switch_cycle[0],Switch_cycle[1]):

    #define time
    t_start= n*15
    t_end = (n+1)*15
    t= np.linspace(t_start,t_end,2)

    #solve equation and save result
    result = scipy.integrate.odeint(Repressor_model, molecules_0, t, args=parameters_list)
    result_all = np.append(result_all,result[1])
    
    #update parameter
    molecules_0 = result.transpose()[:,-1]
    
    #dilution 
    ###replenish 
    molecules_0[R.idx] = R.ic*DR+(1-DR)*molecules_0[R.idx]
    

    ###diute out
    dilute_species((T7_RNA,GFP,GFP_RNA,Repressor,Repressor_RNA),molecules_0)



for n in range (Switch_cycle[1],Switch_cycle[2]):

    #define time
    t_start= n*15
    t_end = (n+1)*15
    t= np.linspace(t_start,t_end,2)

    #solve equation and save result
    result = scipy.integrate.odeint(Repressor_model, molecules_0, t, args=parameters_list)
    result_all = np.append(result_all,result[1])
    #update parameter
    molecules_0 = result.transpose()[:,-1]
    
    #dilution 
    ###replenish 
    molecules_0[R.idx] = R.ic*DR+(1-DR)*molecules_0[R.idx]

    ###diute out
    dilute_species((T7,T7_RNA,T7_DNA,GFP,GFP_RNA,GFP_DNA,Repressor,Repressor_RNA,Repressor_DNA),molecules_0)



result_all = np.resize(result_all,(96,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()


(array([ 1.00000000e+00,  0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  2.00000000e-01,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+01,  1.00000000e+00,  3.64921989e+00,
         1.58008068e+00,  1.00000000e+00,  3.95249846e+00,  6.03685593e-01,
         2.00000000e-01,  7.90499692e-01,  1.20737119e-01, -2.19331196e-10,
         1.00000000e+00,  3.54769188e+00,  1.91454847e+00,  1.00000000e+00,
         3.92961232e+00,  6.30267141e-01,  2.00000000e-01,  7.85922463e-01,
         1.26053428e-01, -2.03943171e-11,  1.00000000e+00,  3.47993876e+00,
         2.22911689e+00,  1.00000000e+00,  3.93078649e+00,  6.32396204e-01,
         2.00000000e-01,  7.86157297e-01,  1.26479241e-01, -8.30377268e-12,
         1.00000000e+00,  3.43807452e+00,  2.52967793e+00,  1.00000000e+00,
         3.94554996e+00,  6.20428017e-01,  2.00000000e-01,  7.89109992e-01,
         1.24085603e-01, -1.64569975e-12,  1.00000000e+00,  3.41622643e+00,
         2.8

In [19]:
Switch_time = np.array([4,16,24])
Switch_cycle = Switch_time*4
print(Switch_cycle)

[16 64 96]


In [20]:
t = np.linspace(0, 1440, 96)
p = bokeh.plotting.figure(
    plot_width=800,
    plot_height=400,
    x_axis_label="t",
    y_axis_type="linear",

)

colors = bokeh.palettes.d3["Category10"][3]


# Populate glyphs
p.line(
    t/60, GFP.result, line_width=2, color=colors[0]
)
vline1 = Span(location=4, dimension='height', line_color='black', line_width=1,line_dash='dashed')
vline2 = Span(location=16, dimension='height', line_color='black', line_width=1,line_dash='dashed')
p.add_layout(vline1)
p.add_layout(vline2)


show(p)


In [14]:
molecules_0=[1,1,1,1,1,1,1,1,1,1]
print(molecules_0)
for i in range(10):
    dilute_species((T7,T7_DNA),molecules_0)
    print(molecules_0)
    replenish_species((T7,T7_DNA),molecules_0)

    print(molecules_0)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.8, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 1.0, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.8, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 1.0, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.8, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 1.0, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.8, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 1.0, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.8, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 1.0, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.8, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 1.0, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.8, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 1.0, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.8, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 1.0, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.8, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 1.0, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.8, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 1.0, 1, 1, 1, 1, 1, 1, 1]
